# Edge 검출 (밝기변화도가 큰 부분을 추출)

* 흑백 처리
  * 오히려 더 적은 edge 검출
  * color이미지는 3채널 각각에 대한 edge를 고려함 -> 정확할 수는 있지만, 쓸데없는 연산이 많아짐
  * 연산량을 줄이기 때문에 사용
  * ps : 추출이 잘 안되면 color로 해보기

* 블러 처리
  * 노이즈와 불필요한 gradient 없애줌
  * 짝수는 안됨

* **Canny 알고리즘**
  * 라인 따줌
  * low 와 high 비율은 1:2 혹은 1:3 추천

In [2]:
import cv2
import numpy as np

def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold): # Canny알고리즘
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kerner_size): # 가우시안 필터
    return cv2.GaussianBlur(img, (kerner_size, kerner_size), 0)

def region_of_interest(img, vertices, color3 = (255, 255, 255), color1 = 255):
    #ROI setting
    mask = np.zeros_like(img) # img와 같은 크기의 빈 이미지

    if len(img.shape) > 2 : #if color image (3 channels)
        color = color3
    else : #if black&white image ( 1 channel)
        color = color1
    
    #vertices에 정한 점들로 이루어진 다각형 부분(ROI부분)을 color로 채움
    cv2.fillPoly(mask, vertices, color)
    
    #이미지와 color로 체워진 ROI를 합침
    ROI_img = cv2.bitwise_and(img, mask)
    return ROI_img

def draw_lines(img, lines, color=[0,0,255], thickness = 2): #선그리기
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap): #허프 변환
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype = np.uint8)
    draw_lines(line_img, lines)
    
    return line_img

def weighted_img(img, initial_img, a = 1, b = 1., c = 0.): #두 이지미 합치기
    return cv2.addWeighted(initial_img, a, img, b, c)
   

In [3]:
image = cv2.imread("slope_test.jpg")
    
height, width = image.shape[:2]

gray_img = grayscale(image) # 흑백이미지로 변환

blur_img = gaussian_blur(gray_img, 3) # Blur 효과

canny_img = canny(blur_img, 70, 210) # Canny Edge 알고리ㅑ

cv2.imshow('canny', canny_img) # Canny 이미지 출력

vertices = np.array([[(50, height), (width/2 - 45, height/2 + 60), (width/2 + 45, height/2 + 60), (width-50, height)]], dtype=np.int32)

roi_img = region_of_interest(canny_img, vertices) # vertices에 정한 점들을 기준으로 ROI이미지 생성
cv2.imshow('roi', roi_img)

hough_img = hough_lines(roi_img, 1, 1 * np.pi/180, 30, 10 ,20) #허프 변환

result = weighted_img(hough_img, image)
cv2.imshow('result', result)
cv2.waitKey(0)


13

: 

# 동영상 적용

In [2]:
cap = cv2.VideoCapture("solidWhiteRight.mp4")
while cap.isOpened():
    ret, image = cap.read()
    if not ret:
        break
    
    height, width = image.shape[:2]

    gray_img = grayscale(image) # 흑백이미지로 변환

    blur_img = gaussian_blur(gray_img, 9) # Blur 효과

    canny_img = canny(blur_img, 70, 210) # Canny Edge 알고리ㅑ

    cv2.imshow('canny', canny_img) # Canny 이미지 출력

    vertices = np.array([[(50, height), (width/2 - 45, height/2 + 60), (width/2 + 45, height/2 + 60), (width-50, height)]], dtype=np.int32)

    roi_img = region_of_interest(canny_img, vertices) # vertices에 정한 점들을 기준으로 ROI이미지 생성
    cv2.imshow('roi', roi_img)

    hough_img = hough_lines(roi_img, 1, 1 * np.pi/180, 30, 10 ,20) #허프 변환

    result = weighted_img(hough_img, image)
    cv2.imshow('result', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release
cap.release()
cv2.destroyAllWindows()
ㄴ

: 